# 온라인 쇼핑몰 데이터 분석 (SQL + Python)
이 노트북은 SQLite + Python을 사용해 가상의 온라인 쇼핑몰 데이터를 생성하고, 매출 및 고객 행동을 분석하는 포트폴리오용 예시입니다.

In [1]:
!pip install faker tqdm

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import random
from faker import Faker
from tqdm import tqdm

# DB 연결
conn = sqlite3.connect('shop.db')
cursor = conn.cursor()

# Faker 설정
fake = Faker('ko_KR')

In [ ]:
create_tables_sql = '''
CREATE TABLE IF NOT EXISTS customers (
    customer_id   INTEGER PRIMARY KEY AUTOINCREMENT,
    gender        TEXT,
    age           INTEGER,
    join_date     DATE,
    region        TEXT
);

CREATE TABLE IF NOT EXISTS products (
    product_id    INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name  TEXT,
    category      TEXT,
    price         INTEGER,
    created_at    DATE
);

CREATE TABLE IF NOT EXISTS orders (
    order_id      INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id   INTEGER,
    order_date    DATETIME,
    order_status  TEXT,
    payment_method TEXT,
    device_type   TEXT
);

CREATE TABLE IF NOT EXISTS order_items (
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id      INTEGER,
    product_id    INTEGER,
    quantity      INTEGER,
    unit_price    INTEGER
);
'''

cursor.executescript(create_tables_sql)
conn.commit()
print('테이블 생성 완료')

테이블 생성 완료


In [ ]:
cursor.executescript('''
DELETE FROM order_items;
DELETE FROM orders;
DELETE FROM products;
DELETE FROM customers;
''')
conn.commit()
print('기존 데이터 초기화 완료')

기존 데이터 초기화 완료


In [ ]:
num_customers = 2000

customers = []
for _ in tqdm(range(num_customers)):
    gender = random.choice(['M', 'F'])
    age = random.randint(18, 65)
    join_date = fake.date_between(start_date='-2y', end_date='today')
    region = random.choice(['Seoul', 'Busan', 'Incheon', 'Daegu', 'Daejeon', 'Gwangju'])
    customers.append((gender, age, join_date, region))

cursor.executemany(
    'INSERT INTO customers (gender, age, join_date, region) VALUES (?, ?, ?, ?)',
    customers
)
conn.commit()
print('고객 데이터 생성 완료:', len(customers), '명')

100%|██████████| 2000/2000 [00:00<00:00, 2569.57it/s]
C:\Users\dlwns\AppData\Local\Temp\ipykernel_14188\3672028637.py:11: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany(


고객 데이터 생성 완료: 2000 명


In [ ]:
num_products = 300
categories = ['패션', '뷰티', '식품', '가전', '생활용품', '스포츠']

products = []
for _ in tqdm(range(num_products)):
    name = fake.word() + '_' + str(random.randint(1, 999))
    category = random.choice(categories)
    price = random.randint(5000, 200000)
    created_at = fake.date_between(start_date='-1y', end_date='today')
    products.append((name, category, price, created_at))

cursor.executemany(
    'INSERT INTO products (product_name, category, price, created_at) VALUES (?, ?, ?, ?)',
    products
)
conn.commit()
print('상품 생성 완료:', len(products), '개')

100%|██████████| 300/300 [00:00<00:00, 1903.16it/s]

상품 생성 완료: 300 개



C:\Users\dlwns\AppData\Local\Temp\ipykernel_14188\3205430828.py:12: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany(


In [ ]:
num_orders = 5000

orders = []
for _ in tqdm(range(num_orders)):
    customer_id = random.randint(1, num_customers)
    order_date = fake.date_time_between(start_date='-1y', end_date='now')
    status = random.choice(['paid', 'paid', 'paid', 'refund'])
    pay = random.choice(['card', 'bank', 'kakao_pay', 'naver_pay'])
    device = random.choice(['web', 'mobile', 'app'])
    orders.append((customer_id, order_date, status, pay, device))

cursor.executemany(
    'INSERT INTO orders (customer_id, order_date, order_status, payment_method, device_type) VALUES (?, ?, ?, ?, ?)',
    orders
)
conn.commit()
print('주문 생성 완료:', len(orders), '건')

100%|██████████| 5000/5000 [00:00<00:00, 5346.91it/s]
C:\Users\dlwns\AppData\Local\Temp\ipykernel_14188\1703068760.py:12: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany(


주문 생성 완료: 5000 건


In [ ]:
order_items = []

for order_id in tqdm(range(1, num_orders + 1)):
    for _ in range(random.randint(1, 3)):
        product_id = random.randint(1, num_products)
        quantity = random.randint(1, 5)
        cursor.execute('SELECT price FROM products WHERE product_id = ?', (product_id,))
        unit_price = cursor.fetchone()[0]
        order_items.append((order_id, product_id, quantity, unit_price))

cursor.executemany(
    'INSERT INTO order_items (order_id, product_id, quantity, unit_price) VALUES (?, ?, ?, ?)',
    order_items
)
conn.commit()
print('주문 상세 생성 완료:', len(order_items), '개')

  0%|          | 0/5000 [00:00<?, ?it/s]


TypeError: 'NoneType' object is not subscriptable

In [ ]:
query_monthly = '''
SELECT
    strftime('%Y-%m', o.order_date) AS ym,
    SUM(oi.quantity * oi.unit_price) AS revenue,
    COUNT(DISTINCT o.order_id) AS num_orders
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = "paid"
GROUP BY ym
ORDER BY ym;
'''

df_month = pd.read_sql_query(query_monthly, conn)
df_month

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(df_month['ym'], df_month['revenue'], marker='o')
plt.xticks(rotation=45)
plt.title('월별 매출 추이')
plt.xlabel('월')
plt.ylabel('매출')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
query_category = '''
SELECT
    p.category,
    SUM(oi.quantity * oi.unit_price) AS revenue
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
JOIN orders o ON oi.order_id = o.order_id
WHERE o.order_status = "paid"
GROUP BY p.category
ORDER BY revenue DESC;
'''

df_cat = pd.read_sql_query(query_category, conn)
df_cat

In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(df_cat['category'], df_cat['revenue'])
plt.title('카테고리별 매출')
plt.xlabel('카테고리')
plt.ylabel('매출')
plt.tight_layout()
plt.show()

In [ ]:
query_gender_age = '''
WITH order_amounts AS (
    SELECT o.order_id, o.customer_id,
           SUM(oi.quantity * oi.unit_price) AS amount
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = "paid"
    GROUP BY o.order_id
)
SELECT c.gender,
       CASE
           WHEN c.age < 20 THEN '10대 이하'
           WHEN c.age BETWEEN 20 AND 29 THEN '20대'
           WHEN c.age BETWEEN 30 AND 39 THEN '30대'
           WHEN c.age BETWEEN 40 AND 49 THEN '40대'
           ELSE '50대 이상'
       END AS age_group,
       COUNT(*) AS num_orders,
       AVG(amount) AS avg_order
FROM order_amounts oa
JOIN customers c ON oa.customer_id = c.customer_id
GROUP BY gender, age_group
ORDER BY age_group;
'''

df_ga = pd.read_sql_query(query_gender_age, conn)
df_ga

In [ ]:
query_repurchase = '''
SELECT customer_id, COUNT(*) AS order_count
FROM orders
WHERE order_status = "paid"
GROUP BY customer_id;
'''

df_re = pd.read_sql_query(query_repurchase, conn)
df_re

In [ ]:
query_device_pay = '''
SELECT
    o.device_type,
    o.payment_method,
    SUM(oi.quantity * oi.unit_price) AS revenue
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = "paid"
GROUP BY o.device_type, o.payment_method
ORDER BY revenue DESC;
'''

df_dp = pd.read_sql_query(query_device_pay, conn)
df_dp